# CHAINS
**TODO:** Replace deprecated code.

In [1]:
import os
import pandas as pd

In [2]:
df = pd.read_csv("resources/data_chains.csv")

In [3]:
df.head(10)

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld,I loved this product. But they only seem to la...
5,L'Or Espresso Café,Je trouve le goût médiocre. La mousse ne tient...
6,Hervidor de Agua Eléctrico,"Está lu bonita calienta muy rápido, es muy fun..."


## 1.- LLM Chain

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [5]:
# Initialize LLM
llm = ChatOpenAI(temperature=0.9, api_key=os.environ["OPENAI_KEY"])

C:\Users\jcav\AppData\Local\Temp\ipykernel_42632\392081221.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0.9, api_key=os.environ["OPENAI_KEY"])


In [6]:
# Create prompt template
prompt = ChatPromptTemplate.from_template("What is the best name to describe a company that makes {product}?") 

In [7]:
# Create chain
chain = LLMChain(llm=llm, prompt=prompt) # Deprecated

C:\Users\jcav\AppData\Local\Temp\ipykernel_42632\573891551.py:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt) # Deprecated


In [8]:
product = "Queen size Sheet set"
chain.run(product) # Deprecated

C:\Users\jcav\AppData\Local\Temp\ipykernel_42632\3167731583.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(product) # Deprecated


'"Royal Linens"'

# 2.- Sequential chains
Sequential chain is another type of change. The idea is to combine multiple chains where the output of one chain is the input of the next chain. 

There are two types of sequential chains:
1. `SimpleSequentialChain`: Single input/output
2. `SequentialChain`: inputs/outputs

### 2.1.- SimpleSequentialChain

In [9]:
from langchain.chains import SimpleSequentialChain

In [10]:
llm = ChatOpenAI(temperature=0.9, api_key=os.environ["OPENAI_KEY"])

# Prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [11]:
# Prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)

# Chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [12]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose=True)

In [13]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
"Royal Comfort Co."
Royal Comfort Co. specializes in luxurious bedding and home essentials, providing high-quality products for a restful and stylish living space.

> Finished chain.


'Royal Comfort Co. specializes in luxurious bedding and home essentials, providing high-quality products for a restful and stylish living space.'

This chain works well when there is a single input and ouput, but we need an alternative when there are multiple inputs and outputs.

### 2.2.- SequentialChain

In [14]:
from langchain.chains import SequentialChain

In [15]:
llm = ChatOpenAI(temperature=0.9, api_key=os.environ["OPENAI_KEY"])

In [16]:
# Prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template("Translate the following review to english: \n\n{review}")

# chain 1, indicate the name of the output
chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="english_review")

In [17]:
# Prompt template 2: Summarize text
second_prompt = ChatPromptTemplate.from_template("Can you summarize the following review in 1 sentence?: \n\n{english_review}")

# chain 2: indicate the output
chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key="summary")

In [18]:
# Prompt template 3: Detect language
third_prompt = ChatPromptTemplate.from_template("What language is the following review:\n\n{review}")

# chain 3: outputs the language
chain_three = LLMChain(llm=llm, prompt=third_prompt, output_key="language")

In [19]:
# Prompt template 4: Write a follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following summary in the specified language: \n\nSummary: {summary}\n\nLanguage: {language}"
)

# chain 4: outputs the follow up message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt, output_key="followup_message")

In [20]:
# Create the overall chain
# input: review
# output: english_review, summary, language, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review"],
    output_variables=["english_review", "summary", "followup_message"],
    verbose=True
)

In [21]:
review = df.Review[5]

overall_chain.invoke(review)



> Entering new SequentialChain chain...

> Finished chain.


{'review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur... Vieux lot ou contrefaçon !?",
 'english_review': "I find the taste mediocre. The foam doesn't hold, it's weird. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'summary': 'The reviewer finds the taste of the product mediocre and suspects it may be an old batch or counterfeit.',
 'followup_message': "Merci pour votre avis honnête. Nous sommes désolés d'apprendre que vous n'avez pas été satisfait du goût du produit. Nous prenons vos préoccupations au sérieux et allons enquêter pour nous assurer que nos produits sont toujours frais et authentiques. Espérons que nous pourrons regagner votre confiance à l'avenir. Merci de nous avoir informés de cette situation."}

## 3.- RouterChain
Directing an input to a chain or another depending on what exactly that input is.

In [22]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [23]:
# These descriptions are passed to the router change, so it can decide which chain to use.
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [24]:
# Used when routing between multiple different prompt templates
from langchain.chains.router import MultiPromptChain

# LLMRouterChain uses an LLM to route between the different subchains. This is where the description and the name provided above will be used.
# RouterOutputParser parses the LLM output into a dictionary that can be used downstream to determine which chain to use and what the input to that chain should be
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser


from langchain.prompts import PromptTemplate

In [27]:
llm = ChatOpenAI(temperature=0.0, api_key=os.environ["OPENAI_KEY"])

In [28]:
# Destination chains are the chains that will be called by the router chain
# Dictionary where key: chain name, value: corresponding LLMChain
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    
    chain = LLMChain(llm=llm, prompt=prompt)
    
    destination_chains[name] = chain

In [30]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [32]:
print(destinations_str)

physics: Good for answering questions about physics
math: Good for answering math questions
History: Good for answering history questions
computer science: Good for answering computer science questions


In [33]:
# We also need a default chain. This chain is called when the router cannot decide which of the subchains to use
default_prompt = ChatPromptTemplate.from_template("{input}")

default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [35]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \\ "DEFAULT" or name of the prompt to use in {destinations}
    "next_inputs": string \\ a potentially modified version of the original input
}}}}
```

REMEMBER: The value of “destination” MUST match one of \
the candidate prompts listed below.\
If “destination” does not fit any of the specified prompts, set it to “DEFAULT.”
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [36]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser()
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [38]:
# Create the overall chain
chain = MultiPromptChain( # Deprecated
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True
)

In [39]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation and emits radiation at all frequencies. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This type of radiation is important in understanding concepts such as thermal radiation and the behavior of objects at different temperatures."

In [40]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The answer to 2 + 2 is 4.'

In [42]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
None: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because DNA carries the genetic information that determines the characteristics and functions of an organism. DNA contains the instructions for building and maintaining an organism, including the proteins that are essential for cell function and structure. This genetic information is passed down from parent to offspring and is essential for the growth, development, and functioning of all cells in the body. Having DNA in every cell ensures that the genetic information is preserved and can be used to carry out the necessary processes for life.'